# Bot for trading BTCUSDT with crossing moving averages


### THIS BOT IS GOING TO OPERATE WITH CROSSING MOVING AVERAGE LINES WHEN:

#### PRICE > MA5 > MA10 > MA20

### Installs

In [1]:
#%pip install python-binance 
#%pip install numpy
#%pip install pandas

### Imports

In [2]:
import config
import math
from binance.client import Client
from binance.enums import *
import time
import numpy as np

cliente = Client(config.API_KEY, config.API_SECRET, tld='com')


# Definition of coin and parameters

In [3]:
simbolo = 'TVKUSDT'   #Name of specific Symbol we are going to trade

simboloBalance = 'TVK'

quantityOrder = 390 # Quantity of BTC I am going to buy per operation. Around 15$...

decimales = '{:.4f}' # Number of decimals in buy parameters, if there is too many decimals depending on the cryptocurrency there will be an error of PRICEFILTER 

# Functions

## ***** MOVING AVERAGES WITH 5 MINS CANDLESTICKS ******



### Moving Average 5 in periods of 5mins during 25 minutes

In [4]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma5_local = sum / 5

    return ma5_local

### Moving Average 10 in periods of 5mins during 50 minutes

In [5]:
def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma10_local = sum / 10

    return ma10_local

### Moving Average 20 in periods of 5mins during 100 minutes

In [6]:
def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma20_local = sum / 20

    return ma20_local

### *** Loop operation for continuous trading ***

In [7]:
while 1:

     ## Calculamos el balance en cuenta para poner orden OCO Exacta y evitar LotSize &/or Insuficent Balance
    
    sum_simbolo = 0.0
    balances = cliente.get_account()
    for _balance in balances["balances"]:
        asset = _balance["asset"]
        if float(_balance["free"]) != 0.0 or float(_balance["locked"]) != 0.0:
            try:
                simbolo_quantity = float(_balance["free"]) + float(_balance["locked"])
                if asset == simboloBalance:
                    sum_simbolo += simbolo_quantity
                else:
                    _price = cliente.get_symbol_ticker(symbol=asset + simboloBalance)
                    sum_simbolo += simbolo_quantity * float(_price["price"])
            except:
                pass
    current_simbolo_price_USD = cliente.get_symbol_ticker(symbol="BTCUSDT")["price"]
    own_usd = sum_simbolo * float(current_simbolo_price_USD)
    print(" Balance en billetera => "  + simboloBalance + " %.8f  == " % sum_simbolo, end="")
    print("")
    
    time.sleep(10)
        
        
       

    

    requestMinQtOrder = cliente.get_symbol_info(simbolo)
    ordenes = cliente.get_open_orders(symbol=simbolo)
    print("Actual open orders") # si devuelve [] esta vacio
    print(ordenes)
    print("")
    

   
    if(len(ordenes) != 0 ):
        print(len(ordenes))
        print("Quantity to sell   " + str(math.floor(sum_simbolo)))
        print("Sell Price if it goes DOWN   " + ordenes[0]['price'])
        print("Sell Price if it goes UP   " + ordenes[1]['price'])
        print("")
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.
        continue

    

    if(len(ordenes) !=0 ):
        print(" There is open orders for this symbol, no new buys")
        print("")
        time.sleep(10)
        continue

    # obtengo el precio del token que estoy tradeando
    list_of_tickers = cliente.get_all_tickers()
    for tick_2 in list_of_tickers:
        if tick_2['symbol'] == simbolo:
            symbolPrice = float(tick_2['price'])
    # fin obtener precio. 

    ma5  = _ma5_()
    ma10 = _ma10_()
    ma20 = _ma20_()

    if(ma20 == 0 ): continue

    requestMinQtOrder = cliente.get_symbol_info(simbolo)

    

    #print("Cantidad minima de ordenes de compra es: " + requestMinQtOrder['filters'][2]['minQty'])


    minQtOrder = float(requestMinQtOrder['filters'][1]['minQty'])

    if (minQtOrder !=1 ):
        print("Order accepts decimals")
        order_local = '{:.8f}'.format(quantityOrder*0.999)
    else:
        print("Order accepts only integer numbers\n")
        order_local = '{:.0f}'.format(quantityOrder*0.999)
    
    # importante acomodar los decimales de la moneda porque arroja Error Price Filter.

    print("--------" + simbolo + "---------")
    print(" Actual price of " + simbolo + "is: " + str(decimales.format(symbolPrice))) #el .8 es la cantidad de decimales que no trae el simbolo 
    print("*******************************")
    print(" Price MA5 " + str(decimales.format(ma5)))
    print(" Price MA10 " + str(decimales.format(ma10)))
    print(" Price MA20 " + str(decimales.format(ma20)))
    print(" Price to buy " + str(decimales.format(ma20*0.995)))
    print("*******************************\n")


    if ( symbolPrice > ma5 and ma5 > ma10 and ma10 > ma20):
        print("Buying if there is no open orders")
        print("")
    
    # ORDENES DE PRUEBA 
        #order = cliente.create_test_order(
        #symbol = simbolo,
        #side = SIDE_BUY,
        #type = ORDER_TYPE_LIMIT,
        #timeInForce = TIME_IN_FORCE_GTC,
        #quantity = cantidadOrden*0.999,
        #price = str(decimales.format(symbolPrice*1.02)),
        #)
#
        #orders = cliente(symbol=simbolo)
        #print(orders)

        order = cliente.order_market_buy(
            symbol = simbolo,
            quantity = quantityOrder
        )
        time.sleep(5)

        #Pongo orden OCO
        print("")
        print("PLACING OCO ORDER")
        print("StopLimitPrice >   " +str(decimales.format(symbolPrice*0.985)),)
        print("Quantity >   " + str(math.floor((sum_simbolo))))
        print("StopPrice >   " + str(decimales.format(symbolPrice*0.99)))
        print("Price >   " + str(decimales.format(symbolPrice*1.01)))
        print("")

        ordenOCO = cliente.create_oco_order(
            symbol = simbolo,
            side = SIDE_SELL,
            stopLimitPrice = str(decimales.format(symbolPrice*0.985)),
            stopLimitTimeInForce = TIME_IN_FORCE_GTC,
            ## Error  LOT SIZE es porque no soporta decimales en quantity
            quantity = str(math.floor((sum_simbolo))), # BINANCE cobra un fee, tarifa. Sino va a tirar un error de insuficent FOUNDS. O Error LOT SIZE. 
            stopPrice = str(decimales.format(symbolPrice*0.99)),
            price = str(decimales.format(symbolPrice*1.01)), 
            )
    
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.

    else: 
        print("Conditions are not met")
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.
    
    # FIN ordenes de prueba

# corregir ma20
# eliminar import BTC primera linea
# corregir filter por FILTERS
# corregir identado en ordenes de prueba

 Balance en billetera => TVK 224.77500000  == 
Actual open orders
[{'symbol': 'TVKUSDT', 'orderId': 88494358, 'orderListId': -1, 'clientOrderId': 'electron_5284c2244f03407b8dcedaa70d3', 'price': '0.04700000', 'origQty': '224.00000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'SELL', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1670930392732, 'updateTime': 1670930392732, 'isWorking': True, 'origQuoteOrderQty': '0.00000000'}]

1
Quantity to sell   224
Sell Price if it goes DOWN   0.04700000


IndexError: list index out of range

# DEBUGGING


In [ ]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma5_local = sum / 5

    return ma5_local

def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma10_local = sum / 10

    return ma10_local

def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma20_local = sum / 20

    return ma20_local

ma5  = _ma5_()
ma10 = _ma10_()
ma20 = _ma20_()

print(ma5 , ma10, ma20)

In [ ]:
requestMinQtOrder = cliente.get_symbol_info(simbolo)

requestMinQtOrder

int(math.floor(float(requestMinQtOrder['filters'][1]['minQty'])))